Import necessary libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image, make_grid
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

Load and preprocess the image from URL

In [ ]:
def load_image_from_url(url, transform):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    return transform(image).unsqueeze(0)

Define the transformation pipeline

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing the image to 224x224
    transforms.ToTensor(),  # Converting the image to Tensor
])

Load the pre-trained VGG19 model

Extract Feature Embeddings

    - Feature embeddings are high-dimensional representations of the image
    - They capture hierarchical information: edges → textures → objects
    - Each layer's output is stored for visualization

In [ ]:
vgg = models.vgg19(pretrained=True).features.eval()

# Extract embeddings from each layer
def get_all_embeddings(image, model):
    embeddings = {}
    for i, layer in enumerate(model):
        image = layer(image)
        embeddings[f'Layer {i}'] = image.detach()
    return embeddings

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 89.2MB/s]


Visualization function for embeddings

Visualizing Feature Embeddings

    - Each feature map consists of multiple channels (depth = number of filters)
    - We compute the **mean activation per channel** to visualize how each layer processes the image
    - Mathematically, the mean activation of a layer `l` is:
      V^l = (1/C) * sum(E^l_c)   where C = number of channels in layer l

In [ ]:
def visualize_embeddings(embeddings):
    for layer, embedding in embeddings.items():
        plt.figure(figsize=(10, 5))
        plt.imshow(embedding[0].mean(0).cpu().numpy(), cmap='viridis')
        plt.title(f'{layer} Embedding Visualization')
        plt.colorbar()
        plt.show()

# Load and process the original image
animal_url = "https://upload.wikimedia.org/wikipedia/commons/7/73/Lion_waiting_in_Namibia.jpg"
original_image = load_image_from_url(animal_url, transform)

# Extract embeddings from all layers
embeddings = get_all_embeddings(original_image, vgg)

# Visualize embeddings
visualize_embeddings(embeddings)

Output hidden; open in https://colab.research.google.com to view.

Summary of the Code: Image Reconstruction Using Embeddings

    This code extracts feature embeddings from a pre-trained VGG19 model and visualizes them.
    Feature embeddings represent high-level features of an image extracted at different layers of a CNN.

Outcome:

    Helps understand how CNN layers process images at different levels.

    Useful for feature visualization & interpretability in deep learning.